In [ ]:
#! If you want to force use of CPU (before import of tensorflow)
import os
# os.environ['CUDA_VISIBLE_DEVICES']='-1'

#! import as tf
import tensorflow as tf

#! If you want to limit number of threads, the mechine has 32 virtual cores (16 physical ones)
print('thread inter = ', tf.config.threading.get_inter_op_parallelism_threads() )
print('thread intra = ', tf.config.threading.get_intra_op_parallelism_threads() )
 
tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)

#! If you want to limit the GPU memory usage, the machine has ~24GB. For most applications a few GBs is enough.
#! If you max out what you specify progressively up the threshhold.
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=6000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

#! Will print out what devices you are using
tf.config.list_physical_devices()



import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

from mlkernels.tensorflow import Matern32, EQ, Matern12, Matern52


from RGNP.NeuralNet import NeuralN
from RGNP.utilities import reparameterize, log_normal_pdf, expHighLowFunc_01
from RGNP.GICNet    import FFLayer, KIPLayer, GICNet

pi  = tf.constant(np.pi, tf.float32)
tfd = tfp.distributions
tfb = tfp.bijectors

dirname='data-phys_data-nonLinDeepONet-PoissonNonLinear1D-diffGrid/'

os.makedirs(dirname, exist_ok=True)


In [ ]:
dimU = 1
dimZ = 5
dimW = 1
dimX = 1

            # x      
domain     = tf.constant( [-1., 1.] )

nxSamples  = 30

zDist = tfd.Uniform(low=tf.constant(-1., shape=[dimZ]), high=tf.constant(1., shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant( 1., shape=[dimW]), high=tf.constant(2., shape=[dimW]))
e_bound = 1e-5
xDist = tfd.Uniform(low=domain[0]+e_bound, high=domain[1]-e_bound) 

def raveled1DDomainGrid(resolution):
    x  = tf.linspace(domain[0], domain[1], resolution[0])
    return x[:, None]


xvect = raveled1DDomainGrid([32])
gridX = xvect[1:-1,:]

print('xGrid.shape = ', gridX.shape)
print('xGrid = ', gridX)

In [ ]:

dirnameData = '../../../fenics/poisson1DNonLinear/data-groundTruth_2--PoissonNonLinear1D_1000'

dataU = tf.constant( np.loadtxt(dirnameData+'/' + 'allGroundTruthU.dat' ), dtype=tf.float32)
dataZ = tf.constant( np.loadtxt(dirnameData+'/' + 'allGroundTruthZ.dat' ), dtype=tf.float32)
dataW = tf.constant( np.loadtxt(dirnameData+'/' + 'allGroundTruthW.dat' ), dtype=tf.float32)
dataX = tf.constant( np.loadtxt(dirnameData+'/' + 'FenicsMeshX.dat'     ), dtype=tf.float32)

sigma_y         = tf.constant(0.05)
nData_input     = 1000
dataY           = ( dataU + tf.random.normal(stddev=sigma_y, shape=dataU.shape) )[:nData_input, 20:-20:3]

batch_size_data = 50

data_single_X     = tf.reshape(dataX, [-1,1])[20:-20:3, :]

print('data_single_X.shape', data_single_X.shape)

dimY            = dataY.shape[1]
nData_y         = dataY.shape[0]
print('dimY', dimY)

dataset_data_YZWX_stack = tf.concat([dataY, dataZ[:nData_input,:], dataW[:nData_input,None]], 1)
print('dataset_data_YZWX_stack.shape', dataset_data_YZWX_stack.shape)

dataset_data_YZWX       = tf.data.Dataset.from_tensor_slices( dataset_data_YZWX_stack )
dataset_data_YZWX_batched = dataset_data_YZWX.batch(batch_size=batch_size_data)

for i in range(5):
    plt.scatter(data_single_X, dataY[i], s=5)
    plt.plot(dataX, dataU[i])
plt.tight_layout()
plt.xlabel('x')
plt.ylabel('y')
# plt.savefig('5Data_ySamples_trueLine_scatterData.pdf')
plt.show()

In [ ]:
# Define fw network

dim_DeepO_out  = dimU

fwNet  = NeuralN('fw', np.array([ 1, 1, dimZ]) )

numFreq = 100

nhfw = 300

fwNet.NN = tf.keras.Sequential([    tf.keras.layers.Dense( nhfw, input_dim=( numFreq * 2 + dimZ + dimW ), activation='swish'),
                                    tf.keras.layers.Dense( nhfw, activation='swish'),
                                    tf.keras.layers.Dense( nhfw, activation='swish'),
                                    tf.keras.layers.Dense( nhfw, activation='swish'),
                                    tf.keras.layers.Dense( nhfw, activation='swish'),
                                    tf.keras.layers.Dense( nhfw, activation='swish'),
                                    tf.keras.layers.Dense( nhfw, activation='swish'),
                                    tf.keras.layers.Dense( dim_DeepO_out ) ])

freqsX = tf.linspace(0., 1., numFreq)
freqs  = expHighLowFunc_01(2., 0.001, freqsX)
tf.print('all freqs FFN \n', freqs, summarize=-1) 
fLayer = [ FFLayer( freqs ) ]

fwNet.lower_alpha = tf.constant(1e-5)
fwNet.upper_w     = tf.constant(1.)

def fwMap(self, z, w, x):

        z_tile  = tf.tile(tf.reshape(z, [1,-1]), [x.shape[0], 1])
        w_tile  = tf.tile(tf.reshape(w, [1,-1]), [x.shape[0], 1])
        xFreq   = fLayer[0]( x )
        x_input = tf.concat([xFreq, z_tile, w_tile], 1)
        mapped  = self.NN( x_input )
        
        return mapped

from types import MethodType
fwNet.call = MethodType(fwMap, fwNet)
fwNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), tf.constant(1., shape=[nxSamples,dimX]))
fwNet.compile()
fwNet.NN.summary()

# fwNet.save_weights(dirname + '/fwModel/fwModel')
# fwNet.load_weights(dirname + '/fwModel/fwModel')

# del fwNet

In [ ]:

def D(x):
    return tf.cos(0.5 * pi * x)

epsilon_r = tf.Variable(1e-2, trainable=False)

Nz = tf.range(0, dimZ, dtype=tf.float32)

def cheby(n, x):
    return tf.cos( n * tf.math.acos(x) )

def bound( xd ): return cheby(Nz, xd)

def kappa(z, u, x):
    Txn  =  tf.vectorized_map(bound, x)
    return tf.math.log( 1. + tf.exp( tf.reshape(u, [-1,1]) * tf.reshape(Txn @ tf.reshape(z, [-1,1]), [-1,1]) ) ) + 0.1


x = tf.linspace(-1., 1., 100)[:, None]
z = zDist.sample()
plt.plot(tf.linspace(-1., 1., 100), kappa(z, x**0., x) )
plt.show()
plt.close()

def forcing(w, x):
    return tf.reshape(w, [-1]) + x * 0.


def residualFunction( zw ):
    
    # x, t = xrand[:,:1], xrand[:, 1:]
    print('zw', zw)
    z, w = zw[ :dimZ ], zw[ dimZ:dimZ+dimW ]
    z    = tf.reshape(z, [-1,1])

    with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t1:
        t1.watch( gridX )

        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t2:
            t2.watch( gridX )

            outfw = fwNet( z, w, gridX )

            u = D( gridX ) * outfw

            k = kappa(z, u, gridX)
            
        u_x = tf.reshape(t2.gradient(u, gridX), [-1,1])
        k_x = tf.reshape(t2.gradient(k, gridX), [-1,1])
    
    u_xx = tf.reshape(t1.gradient(u_x, gridX), [-1,1])

    f = forcing(w, gridX)
    
    res = tf.reduce_mean( (k_x * u_x + k * u_xx + f)**2. )
        
    return res


z     =  zDist.sample(  ) 
w     =  wDist.sample(  ) 
print('z', z)
print('w', w)
residualFunction( tf.concat([z,w],0) )

In [ ]:
def res_data( YZW ):

    y, z, w = YZW[:dimY], YZW[dimY:dimY+dimZ], YZW[dimY+dimZ:]
    y       = tf.reshape(y, [-1,1])

    print('y.shape', y.shape)
    print('z.shape', z.shape)
    print('w.shape', w.shape)


    outfw = fwNet( z, w, data_single_X )

    Dvect = D( data_single_X )

    outfw_trans = Dvect * outfw

    return  tf.reduce_mean(  (tf.reshape(y, shape=outfw_trans.shape) - outfw_trans)**2. )

res_data(dataset_data_YZWX_stack[0,:])

In [ ]:
trainableVarsfw   = [ fwNet.trainable_variables ]

@tf.function
def train_step_fw(optimizer, batch_phys, batch_data):

    with tf.GradientTape(persistent=True) as tape:
    
        losses_phys = tf.vectorized_map( residualFunction, batch_phys )
        loss_phys = tf.reduce_mean(losses_phys)
        
        losses_data = tf.vectorized_map( res_data, batch_data )
        loss_data   = nData_y / batch_size_data * tf.reduce_sum(losses_data)
        
        print('loss_data = ', loss_data)
        
        loss = loss_phys + loss_data

    for vars in trainableVarsfw:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))

    return loss, loss_data, loss_phys

batchSize = 50
nData     = 1_000
datasetZ  = zDist.sample( nData )
datasetW  = wDist.sample( nData )
datasetZW = tf.concat([datasetZ, datasetW], 1)
dataset_phys        = tf.data.Dataset.from_tensor_slices( datasetZW )
datasetBatched_phys = dataset_phys.batch(batch_size=batchSize)

num_iterations   = tf.constant(1_000_000)
epochs           = int(num_iterations / nData)

intervalEPOCHsave = 100


lr = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, int(num_iterations/10), 0.5, staircase=True, name=None
)
optimizer = tf.keras.optimizers.Adam( lr )

lossfw_all     = []
startTrainingfw = time.time()

iteration = tf.constant(0)
for epoch in tf.range( epochs ):
    start_time = time.time()
    for batch_phys, batch_data in zip(datasetBatched_phys, dataset_data_YZWX_batched):
        iteration += batchSize
        loss, loss_data, loss_phys = train_step_fw(optimizer, batch_phys, batch_data )
    if iteration >= num_iterations: 
        print('DONE BREAK')
        break

    end_time = time.time()

    if epoch % intervalEPOCHsave == 0:
        lossfw_all.append( -loss ) 
        print('Iteration: {}, Epoch: {}, ELBO: {:.2f}, loss_phys: {:.2f}, loss_data: {:.2f}, time elapse for current epoch: {:.2f}'
           .format(iteration, epoch, -loss, -loss_phys, -loss_data, end_time - start_time))

timeTrainfw = (time.time() - startTrainingfw)/60
print('totalTimeTraining = {:.2f}m'.format(timeTrainfw))


In [ ]:
lossfw_all = tf.stack(lossfw_all)
LOSS_fw    = np.array(lossfw_all)
plt.plot(range(LOSS_fw.shape[0]), LOSS_fw )
plt.show()
print(np.isnan(LOSS_fw).mean())

In [ ]:
# Define bw network

dim_trunc_in_bw   = dimW
dim_trunc_out_bw  = 100 

dim_branch_in_y_bw = 1
dim_branch_P_bw    = 20
dv_bw              = 20 # number of channels -- filters
dim_branch_out_bw  = 100 #dimZ #20 #works


dim_DeepO_in_bw   = dim_trunc_out_bw + dim_branch_out_bw
dim_DeepO_out_bw  = dimZ

kip_res       = tf.constant([20])
grid_kip_bw   = raveled1DDomainGrid( kip_res )

gicNetbw = GICNet( grid_kip_bw, kip_res, dim_branch_in_y_bw, dim_branch_P_bw, dimX  )

gicNetbw.P_kLayers   = [ tf.keras.layers.Dense( dim_branch_P_bw, input_dim=( dim_branch_P_bw ), activation ='swish'),
                           tf.keras.layers.Dense( dim_branch_P_bw, activation ='swish'),
                           tf.keras.layers.Dense( dim_branch_P_bw, activation ='swish'),
                           tf.keras.layers.Dense( dim_branch_P_bw, activation ='swish') ]
                                 
gicNetbw.kipLayers   = []
ellXs = tf.linspace(0., 1., dim_branch_P_bw)
low  = 1e-3
high = 1.
ellVals     = expHighLowFunc_01(low, high, ellXs)
for i in range(dim_branch_P_bw):
        gicNetbw.kipLayers.append(KIPLayer(EQ(), ellVals[i]))
        print('kip layer {} ell = '.format(i), gicNetbw.kipLayers[i].ell)

gicNetbw.FFMlayer    = [ ]

gicNetbw.preTrunclayers  = [ 
                         tf.keras.layers.Dense( dim_trunc_out_bw, input_dim=( dim_trunc_in_bw ), activation ='swish'), 
                         tf.keras.layers.Dense( dim_trunc_out_bw,                                  activation ='swish'), 
                         tf.keras.layers.Dense( dim_trunc_out_bw,                                  activation ='swish')
                         ]

gicNetbw.convLayers = [ 
                        tf.keras.layers.Conv1D( dv_bw, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_bw, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_bw, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_bw, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_bw, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Conv1D( dv_bw, (3), strides=(1), activation='swish' ),
                        tf.keras.layers.Flatten(  ),
                        tf.keras.layers.Dense(500),
                        tf.keras.layers.Dense(dim_branch_out_bw),
                        ]
                        
# nn  = tf.keras.Sequential( gicNetbw.convLayers  )
# val = tf.random.uniform(shape=[1,kip_res[0], dim_branch_P_bw + dimX])
# nn(val)
# nn.summary()
# nn = None

nhbw       = 300
deep0_bw   = tf.keras.layers.Dense( nhbw, input_dim=( dim_DeepO_in_bw ), activation=tf.keras.activations.swish)
deep1_bw   = tf.keras.layers.Dense( nhbw, activation=tf.keras.activations.swish)
deep2_bw   = tf.keras.layers.Dense( nhbw, activation=tf.keras.activations.swish)
deep3_bw   = tf.keras.layers.Dense( nhbw, activation=tf.keras.activations.swish)
deep2_bw   = tf.keras.layers.Dense( nhbw, activation=tf.keras.activations.swish)
deep3_bw   = tf.keras.layers.Dense( nhbw, activation=tf.keras.activations.swish)
deepOut_bw = tf.keras.layers.Dense( dim_DeepO_out_bw )
gicNetbw.DeepOlayers = [ deep0_bw, deep1_bw, deep2_bw, deep3_bw, deepOut_bw ]


bwNet    = NeuralN('bw', np.array([ 1, 1, dimZ]) )
bwNet.NN = gicNetbw

bwNet.lower_alpha  = tf.constant(1e-5)
bwNet.upper_w      = tf.constant(1.)

def bwMap(self, u, w, x):

        trunc_in  = tf.reshape(w, [-1,dimW])
        branch_in = tf.concat([x, u], 1)

        mapped = self.NN( branch_in, trunc_in )
        print('mapped.shape = ', mapped.shape)

        tf.debugging.assert_all_finite(mapped, 'mapped not finite')

        return mapped

from types import MethodType
bwNet.call = MethodType(bwMap, bwNet)

outfw = fwNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), gridX)
print(outfw.shape)

bwNet( outfw, tf.constant(1., shape=[dimW]), gridX )
bwNet.compile( )
bwNet.summary( )

# bwNet.save_weights(dirname + '/bwModel/bwModel')
# bwNet.load_weights(dirname + '/bwModel/bwModel')

# del bwNet


In [ ]:
def bind(zw):
    z, w = zw[ :dimZ ], zw[ dimZ:dimZ+dimW ]
    return tf.squeeze( fwNet( z, w, gridX ))    

datasetUfw = tf.vectorized_map(bind, datasetZW )
print(datasetUfw.shape)

datasetUWZ        = tf.concat([datasetUfw, datasetW, datasetZ], 1)
datasetUWZBatched =  tf.data.Dataset.from_tensor_slices( datasetUWZ ).batch(batch_size=batchSize)

In [ ]:
def bwLoss( u_untranswz ):
    u_untrans, w, z = u_untranswz[ :nxSamples ], u_untranswz[ nxSamples:nxSamples+dimW ], u_untranswz[nxSamples+dimW:]
    print('u_untrans.shape', u_untrans)
    print('w.shape', w)
    print('z.shape', z)
    predbw    = bwNet( tf.reshape(u_untrans, [-1,1]), w , gridX)
    loss      = tf.reduce_mean(  (tf.reshape( predbw, [-1] ) - tf.reshape( z, [-1] ))**2. )
    return loss

In [ ]:
trainableVarsbw   = [ bwNet.trainable_variables ]

@tf.function
def train_step_bw(optimizer, batch):

    with tf.GradientTape(persistent=True) as tape:

        losses = tf.vectorized_map( bwLoss, batch )
        loss   = tf.reduce_mean( losses )
    for vars in trainableVarsbw:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))

    return loss

num_iterations_bw = num_iterations

intervalEPOCHsave = 10

lr_bw = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, int(num_iterations_bw/10), 0.5, staircase=True, name=None
)
optimizer_bw = tf.keras.optimizers.Adam( lr_bw )

lossAll_bw     = []
startTrainingDeepO_bw = time.time()

iteration_bw = tf.constant(0)
startTrainingbw = time.time()
for epoch in tf.range( epochs ):
    start_time = time.time()
    for batch in datasetUWZBatched:
        iteration_bw += batchSize
        loss = train_step_bw(optimizer_bw, batch)
    if iteration_bw >= num_iterations_bw: 
        print('DONE BREAK')
        break

    end_time = time.time()

    if epoch % intervalEPOCHsave == 0:
        lossAll_bw.append( -loss ) 
        print('Iter: {}, Epoch: {}, Test set loss: {}, time elapse for current epoch: {:.2f}'
           .format(iteration_bw, epoch, -loss, end_time - start_time))


timeTrainbw = (time.time() - startTrainingbw)/60
print('totalTimeTraining = {:.2f}m'.format(timeTrainbw))


In [ ]:
lossAll_bw = tf.stack(lossAll_bw)
LOSS_bw    = np.array(lossAll_bw)
plt.plot(range(LOSS_bw.shape[0]), LOSS_bw )
plt.show()
print(np.isnan(LOSS_bw).mean())

In [ ]:
print('totTimetraining = ', timeTrainfw + timeTrainbw)

dirnameGT = '../../../fenics/poisson1DNonLinear/data-groundTruth--PoissonNonLinear1D_1000'


groundTruthU = tf.constant( np.loadtxt(dirnameGT+'/' + 'allGroundTruthU.dat' ), dtype=tf.float32)
allTrueZs    = tf.constant( np.loadtxt(dirnameGT+'/' + 'allGroundTruthZ.dat' ), dtype=tf.float32)
allTrueWs    = tf.constant( np.loadtxt(dirnameGT+'/' + 'allGroundTruthW.dat' ), dtype=tf.float32)
groundTruthX = tf.constant( np.loadtxt(dirnameGT+'/' + 'FenicsMeshX.dat'     ), dtype=tf.float32)

saveDataList = True

if saveDataList:

     allresAbsMean = []

     allPredZs     = []

     allUPreds     = []

     allZNSE       = []
     allUNSE       = []

PLOT = False

Xtest   = groundTruthX[:, None]

tf_residualFunction = tf.function( residualFunction )
tf_fwNet = tf.function( fwNet )
tf_bwNet  = tf.function( bwNet )

for i in range(groundTruthU.shape[0]):

     zTrue = allTrueZs[i]
     wTrue = allTrueWs[i][None]
     uTrue = groundTruthU[i]
     
     residual = tf_residualFunction( tf.concat([zTrue, wTrue], 0) )
     u_untrans = tf_fwNet( zTrue, wTrue, Xtest )

     u = tf.squeeze( D( Xtest ) * u_untrans )

     if PLOT:
          c = next(plt.gca()._get_lines.prop_cycler)['color']

          plt.plot(Xtest, u,  '--', linewidth=1,  c=c)
          plt.plot(groundTruthX[:, None], uTrue,   linewidth=1,    c=c)
     
     
     z = tf.reshape( tf_bwNet( tf.reshape(uTrue/D(tf.squeeze(Xtest)), [-1,1]), wTrue,  Xtest ), [-1] )

     print('z = ', z)
     print('w = ', wTrue)
     print('true z = '   , zTrue)
     print('\n\n')


     if saveDataList:

          allresAbsMean.append( tf.squeeze(residual) ) 

          allUPreds.append( tf.squeeze(u) )

          allPredZs.append( tf.squeeze(z) )
          
          allZNSE.append( ( tf.linalg.norm((tf.squeeze(z) - tf.squeeze(zTrue)) ) / tf.linalg.norm(tf.squeeze(zTrue)) )**2. )

          allUNSE.append( ( tf.linalg.norm((tf.squeeze(u) - tf.squeeze(uTrue)) ) / tf.linalg.norm(tf.squeeze(uTrue)) )**2. )

if PLOT:

     plt.xlabel(r'$x$')
     plt.ylabel(r'$u(x)$')
     plt.tight_layout()
     plt.show()


print('mean allZNSE = ', tf.reduce_mean( tf.stack(allZNSE) ) )
print('mean allUNSE = ', tf.reduce_mean( tf.stack(allUNSE) ) )
print('sttdev allZNSE = ', tf.math.reduce_std( tf.stack(allZNSE) ) )
print('sttdev allUNSE = ', tf.math.reduce_std( tf.stack(allUNSE) ) )
print('totTimetraining = ', timeTrainfw + timeTrainbw)
raise

In [ ]:
if True:
    fwNet.save_weights(dirname + '/fwModel/fwModel')
    # alphaNet.load_weights(dirname + '/fwModel/fwModel')

    # del alphaNet
if True:
    bwNet.save_weights(dirname + '/bwModel/bwModel')
    # betaNet.load_weights(dirname + '/bwModel/bwModel')

    # del betaNet

In [ ]:
saveData = True
if saveData:

    prefix = 'randgrid{}'.format(nxSamples)

    np.savetxt(dirname + prefix + '_allRes_np.dat'   , tf.stack(tf.squeeze(allresAbsMean)) )

    np.savetxt(dirname + prefix + '_allPredZs.dat', tf.stack(allPredZs))

    np.savetxt(dirname + prefix + '_allUPreds.dat' , tf.stack(allUPreds))

    np.savetxt(dirname + prefix + '_allZNSE.dat'   , tf.stack( allZNSE ) )
    np.savetxt(dirname + prefix + '_allUNSE.dat'   , tf.stack( allUNSE ) )

In [ ]:
for layer in bwNet.NN.kipLayers:
    print(layer.ell)